In [ ]:
#@title 🎧 Download Narration Audio & Play Introduction
import os as _os
if not _os.path.exists("/content/narration"):
    !pip install -q gdown
    import gdown
    gdown.download(id="142TELN2j0jMWSJ94WD61z2eEAaja4ur-", output="/content/narration.zip", quiet=False)
    !unzip -q /content/narration.zip -d /content/narration
    !rm /content/narration.zip
    print(f"Loaded {len(_os.listdir('/content/narration'))} narration segments")
else:
    print("Narration audio already loaded.")

from IPython.display import Audio, display
display(Audio("/content/narration/00_intro.mp3"))

In [ ]:
#@title 🎧 Listen: Setup & Libraries
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_setup.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

# 🚀 World Models from First Principles
## Notebook 1 of 6 — *World Action Models: How AI Learns to Imagine Before It Acts*

**Vizuara AI — Google Colab Notebook**

In this notebook, we will build a world model from scratch. Not a toy example — a real neural network that learns the physics of CartPole and uses that learned physics to make intelligent decisions.

**Estimated time: ~25 minutes**

**What you will learn:**
- Why imagining the future is the key to intelligent behavior
- The formal mathematics behind world models
- How to build, train, and use a neural network world model in PyTorch

In [ ]:
# Setup — run this cell first
!pip install gymnasium -q

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import gymnasium as gym

# Reproducibility
torch.manual_seed(42)
np.random.seed(42)

%matplotlib inline

print("All libraries loaded successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"Gymnasium version: {gym.__version__}")

In [ ]:
#@title 🎧 Listen: Why World Models Matter
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_why_it_matters.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 1. Why Does This Matter?

Let us start with something you do every single day without thinking about it.

Right now, imagine reaching out and picking up a glass of water from your desk. Before your hand even moves, your brain has already **simulated** the entire action:

- How far is the glass? Your brain estimates the distance.
- How heavy will it be? Your brain predicts the weight based on how full it looks.
- What angle should your fingers approach at? Your brain plans the grasp.
- What happens if you grip too loosely? Your brain imagines the glass slipping.

All of this happens in a fraction of a second, **before you move a single muscle.** You are running a simulation — a *world model* — inside your head.

Now here is the remarkable part. A toddler learning to pick up a glass does not need to drop it 10,000 times to learn. They watch, they think, they imagine, and they try — maybe fumbling a few times. Compare this to how most AI agents learn today: through millions of blind trial-and-error interactions.

> The core idea behind world models is simple: **what if we gave AI the ability to imagine before it acts?**

By the end of this notebook, you will build a world model that can predict CartPole physics without ever seeing the environment — and use that model to make intelligent decisions through pure imagination.

In [ ]:
#@title 🎧 Listen: Building Intuition
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_building_intuition.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

In [ ]:
#@title 🎧 Listen: Think About It
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_think_about_it.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 2. Building Intuition

Before we write any code or equations, let us build a mental model for what a world model actually does. We will compare two fundamentally different approaches to learning.

### Model-Free Learning: Learning by Crashing

Imagine you are learning to drive a car, but the *only* way you are allowed to learn is by getting into a real car on a real road:

- You press the gas. The car lurches forward. You crash into a wall. **Negative reward.**
- You try again. This time you turn the wheel slightly. You swerve off the road. **Negative reward.**
- After 50,000 crashes, you start to develop reflexes. Gas pedal here, brake there, turn the wheel this much at that speed.

You have learned a **policy** — a mapping from situations to actions — but you have learned it entirely through pain. You never understood *why* pressing the gas makes the car accelerate. You just learned the pattern: "when I see this, do that."

This is **model-free reinforcement learning.** It works, but it is brutally inefficient.

### Model-Based Learning: The Driving Simulator

Now imagine a different approach. Before you ever touch a real car, someone gives you a **driving simulator:**

- You can try any action and instantly see what *would* happen.
- You can rewind and try a different action.
- You can simulate 1,000 scenarios in the time it takes to drive one mile.

With the simulator, you learn the **physics** of driving — how steering angle affects trajectory, how speed affects braking distance. You build an internal model of how the world works, and then you use that model to plan your actions.

This is **model-based reinforcement learning.** The simulator is your world model.

### 🤔 Think About This

Before we continue, consider this thought experiment:

- A model-free agent needs to interact with the real environment for every single learning step. Each interaction costs time, energy, and potentially real-world consequences.
- A model-based agent can "imagine" thousands of interactions inside its head. The only real-world interactions it needs are to improve its internal model.

Which approach would you want for a robot surgeon? A self-driving car? A Mars rover that cannot afford a single mistake?

This is exactly why world models matter. They enable **sample-efficient learning** — achieving good performance with far fewer real-world interactions.

In [ ]:
#@title 🎧 Listen: The Core Equation
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_math_core_equation.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 3. The Mathematics

Now let us formalize these intuitions. We will build up the mathematics step by step, and after every equation, we will explain exactly what it means computationally.

### 3.1 State Transitions: The Core Equation

At the heart of every world model is a single idea: **given where I am now and what I do, where will I be next?**

Mathematically, we write this as:

$$s_{t+1} = f(s_t, a_t)$$

Let us break this down:
- $s_t$ is the **state** at time $t$ — everything the agent knows about the world right now. In CartPole, this is a vector of 4 numbers: cart position, cart velocity, pole angle, and pole angular velocity.
- $a_t$ is the **action** the agent takes — in CartPole, this is either 0 (push left) or 1 (push right).
- $f$ is the **transition function** — the "physics engine" that determines what happens next.
- $s_{t+1}$ is the **next state** — where the world ends up after the action.

Computationally, this means: we take a vector (the current state) and an integer (the action), concatenate them, feed them through a neural network, and get back a new vector (the predicted next state). That is it. The neural network *is* our world model.

### A Numerical Example

Let us plug in some concrete numbers to see how this works. Suppose at time $t = 0$:

- $s_0 = [0.02, 0.15, -0.03, 0.08]$ — the cart is slightly right of center, moving right, with the pole tilted slightly left
- $a_0 = 1$ — we push right

Our world model $f$ takes this input and predicts:

- $s_1 = [0.05, 0.34, -0.02, -0.15]$ — the cart moved further right, sped up, and the pole angle changed

The question is: how close is this prediction to what the *real* physics would produce? That is exactly what we will train our model to get right.

In [ ]:
#@title 🎧 Listen: Reward Prediction
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/06_math_reward.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### 3.2 Reward Prediction

A world model that only predicts states is useful, but a *complete* world model also predicts **rewards.** The reward tells the agent whether what happened was good or bad.

$$\hat{r}_t = g(s_t, a_t)$$

Here:
- $\hat{r}_t$ is the **predicted reward** at time $t$ (the hat symbol $\hat{}$ means "predicted" or "estimated")
- $g$ is a **reward prediction function** — another neural network
- $s_t$ and $a_t$ are the current state and action, just as before

Computationally, this means: we take the same state-action input, feed it through a *second* neural network, and get back a single number — the predicted reward. In CartPole, the real reward is +1 for every step the pole stays upright, and 0 when the episode ends.

### A Numerical Example

Using our earlier numbers, if $s_0 = [0.02, 0.15, -0.03, 0.08]$ and $a_0 = 1$:

- The real reward is $r_0 = 1.0$ (the pole has not fallen)
- Our reward predictor outputs $\hat{r}_0 = 0.97$

The error is $|1.0 - 0.97| = 0.03$, which is quite small. This tells us our reward model has learned that as long as the pole angle is small and the cart is near center, the reward is close to 1.

In [ ]:
#@title 🎧 Listen: The Planning Loop
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/07_math_planning.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### 3.3 The Model-Based Planning Loop

Now the question is: once we have a world model, **how do we use it to make decisions?**

The answer is planning. We simulate multiple possible futures inside our model, and then pick the action sequence that leads to the best outcome.

Here is the planning loop, step by step:

**Step 1: Sample action sequences.** Generate $N$ random action sequences, each of length $H$ (the planning horizon):

$$A^{(i)} = [a_0^{(i)}, a_1^{(i)}, \ldots, a_{H-1}^{(i)}] \quad \text{for } i = 1, \ldots, N$$

**Step 2: Roll forward through the world model.** For each action sequence, use the world model to predict the trajectory:

$$s_{t+1}^{(i)} = f(s_t^{(i)}, a_t^{(i)})$$
$$\hat{r}_t^{(i)} = g(s_t^{(i)}, a_t^{(i)})$$

Computationally, this means: start from the current real state, feed it into our neural network along with the first action, get the predicted next state, then feed *that* predicted state along with the second action, and so on. We are "imagining" an entire trajectory.

**Step 3: Evaluate each sequence.** Sum up the predicted rewards for each trajectory:

$$R^{(i)} = \sum_{t=0}^{H-1} \hat{r}_t^{(i)}$$

**Step 4: Pick the best one.** Select the action sequence with the highest total predicted reward:

$$A^* = A^{(\arg\max_i R^{(i)})}$$

**Step 5: Execute the first action.** Take only the first action $a_0^*$ from the best sequence. Then re-plan from the new real state.

This is called **random shooting** — we randomly "shoot" many trajectories into the future and keep the best one. It is simple, but remarkably effective. Let us now build this entire pipeline.

In [ ]:
#@title 🎧 Listen: Collecting Training Data
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/08_data_collection.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 4. Let Us Build It

Enough theory — let us get our hands dirty. We will build a complete world model for CartPole in four steps:

1. Collect training data from the real environment
2. Build the world model architecture
3. Train it to predict next states
4. Test its predictions against reality

### 4.1 Collecting Training Data

First, we need examples of how CartPole actually behaves. We will run a random policy (taking random actions) and record every transition: $(s_t, a_t) \rightarrow s_{t+1}$.

In [ ]:
def collect_trajectories(env_name="CartPole-v1", num_episodes=200):
    """
    Collect state transitions by running a random policy.

    Returns:
        states: array of current states
        actions: array of actions taken
        next_states: array of resulting next states
        rewards: array of rewards received
    """
    env = gym.make(env_name)

    states, actions, next_states, rewards = [], [], [], []

    for episode in range(num_episodes):
        state, _ = env.reset(seed=episode)
        done = False

        while not done:
            action = env.action_space.sample()  # Random action
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated

            states.append(state)
            actions.append(action)
            next_states.append(next_state)
            rewards.append(reward)

            state = next_state

    env.close()
    return (np.array(states), np.array(actions),
            np.array(next_states), np.array(rewards))

# Collect data
states, actions, next_states, rewards = collect_trajectories()
print(f"Collected {len(states)} transitions from 200 episodes")
print(f"State shape: {states.shape}")
print(f"Example state: {states[0]}")
print(f"Example action: {actions[0]}")
print(f"Example next state: {next_states[0]}")

Let us take a moment to understand what we just collected. Each transition is a single step of CartPole physics: we were in state $s_t$, we took action $a_t$, and the environment moved to state $s_{t+1}$.

The state has 4 dimensions:

| Index | Meaning | Typical Range |
|-------|---------|---------------|
| 0 | Cart position | -2.4 to 2.4 |
| 1 | Cart velocity | -3.0 to 3.0 |
| 2 | Pole angle (radians) | -0.21 to 0.21 |
| 3 | Pole angular velocity | -3.0 to 3.0 |

Our world model needs to learn the relationship between these 4 numbers + the action and predict what the next 4 numbers will be. This is a regression problem.

In [ ]:
# 📊 Let us visualize the data we collected
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
labels = ['Cart Position', 'Cart Velocity', 'Pole Angle', 'Pole Angular Velocity']

for i, (ax, label) in enumerate(zip(axes.flat, labels)):
    ax.hist(states[:, i], bins=50, alpha=0.7, color='steelblue', edgecolor='black')
    ax.set_title(label, fontsize=13)
    ax.set_xlabel('Value')
    ax.set_ylabel('Count')

plt.suptitle('Distribution of Collected States', fontsize=15, fontweight='bold')
plt.tight_layout()
plt.show()

print(f"Action distribution: {np.bincount(actions)} (roughly 50/50 — as expected for random)")

In [ ]:
#@title 🎧 Listen: World Model Architecture
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/09_world_model_architecture.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### 4.2 Building the World Model

Now let us build the core of our system: a neural network that takes $(s_t, a_t)$ and predicts $s_{t+1}$.

We will use a simple feedforward network with two hidden layers. The key design choice is that the input is the state (4 values) concatenated with the action (1 value), giving us 5 input features. The output is the predicted next state (4 values).

In [ ]:
class WorldModel(nn.Module):
    """
    A neural network that learns environment dynamics.

    Given the current state and action, predicts the next state.
    Architecture: Linear(5, 128) -> ReLU -> Linear(128, 128) -> ReLU -> Linear(128, 4)
    """

    def __init__(self, state_dim=4, action_dim=1, hidden_dim=128):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(state_dim + action_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, state_dim)
        )

    def forward(self, state, action):
        """
        Predict the next state.

        Args:
            state: (batch, 4) current state
            action: (batch, 1) action taken
        Returns:
            next_state: (batch, 4) predicted next state
        """
        # Concatenate state and action into a single input vector
        x = torch.cat([state, action], dim=-1)
        return self.network(x)

# Create the model
world_model = WorldModel()
print(world_model)
print(f"\nTotal parameters: {sum(p.numel() for p in world_model.parameters()):,}")

In [ ]:
#@title 🎧 Listen: Training the World Model
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/10_training.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

Notice how simple this is. The entire world model is just three linear layers with ReLU activations. There is no magic here — the network learns to approximate the function $f(s_t, a_t) = s_{t+1}$ from data. The 128 hidden units give it enough capacity to capture the nonlinear physics of CartPole without overfitting.

### 4.3 Training the World Model

Now we train the model. The loss function is straightforward — mean squared error (MSE) between the predicted next state and the actual next state:

$$\mathcal{L} = \frac{1}{N} \sum_{i=1}^{N} \|f(s_t^{(i)}, a_t^{(i)}) - s_{t+1}^{(i)}\|^2$$

Computationally, this means: for each training example, we feed the state and action through the network, get a predicted next state, compute the squared difference with the real next state across all 4 dimensions, and average over the batch.

In [ ]:
def prepare_data(states, actions, next_states):
    """Convert numpy arrays to PyTorch tensors and create a simple dataset."""
    X_state = torch.FloatTensor(states)
    X_action = torch.FloatTensor(actions).unsqueeze(-1)  # Shape: (N, 1)
    Y = torch.FloatTensor(next_states)

    # Train/validation split (90/10)
    n = len(X_state)
    split = int(0.9 * n)
    idx = torch.randperm(n)

    train_idx, val_idx = idx[:split], idx[split:]
    return (X_state[train_idx], X_action[train_idx], Y[train_idx],
            X_state[val_idx], X_action[val_idx], Y[val_idx])

# Prepare data
(train_states, train_actions, train_targets,
 val_states, val_actions, val_targets) = prepare_data(states, actions, next_states)

print(f"Training samples: {len(train_states)}")
print(f"Validation samples: {len(val_states)}")

In [ ]:
def train_world_model(model, train_states, train_actions, train_targets,
                      val_states, val_actions, val_targets,
                      epochs=100, batch_size=256, lr=1e-3):
    """
    Train the world model using MSE loss.

    Returns lists of training and validation losses for plotting.
    """
    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.MSELoss()

    train_losses, val_losses = [], []
    n_train = len(train_states)

    for epoch in range(epochs):
        # Shuffle training data each epoch
        perm = torch.randperm(n_train)
        epoch_loss = 0.0
        n_batches = 0

        model.train()
        for i in range(0, n_train, batch_size):
            idx = perm[i:i+batch_size]
            pred = model(train_states[idx], train_actions[idx])
            loss = loss_fn(pred, train_targets[idx])

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            n_batches += 1

        # Validation
        model.eval()
        with torch.no_grad():
            val_pred = model(val_states, val_actions)
            val_loss = loss_fn(val_pred, val_targets).item()

        train_losses.append(epoch_loss / n_batches)
        val_losses.append(val_loss)

        if (epoch + 1) % 20 == 0:
            print(f"Epoch {epoch+1:3d} | Train Loss: {train_losses[-1]:.6f} | Val Loss: {val_loss:.6f}")

    return train_losses, val_losses

# Train the model
print("Training world model...")
train_losses, val_losses = train_world_model(
    world_model, train_states, train_actions, train_targets,
    val_states, val_actions, val_targets
)
print("\nTraining complete!")

In [ ]:
# 📊 Visualization: Training and Validation Loss
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss', color='steelblue', linewidth=2)
plt.plot(val_losses, label='Validation Loss', color='coral', linewidth=2)
plt.xlabel('Epoch', fontsize=13)
plt.ylabel('MSE Loss', fontsize=13)
plt.title('World Model Training Progress', fontsize=15, fontweight='bold')
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.yscale('log')
plt.tight_layout()
plt.show()

print(f"Final training loss: {train_losses[-1]:.6f}")
print(f"Final validation loss: {val_losses[-1]:.6f}")

In [ ]:
#@title 🎧 Listen: Testing Predictions
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/11_testing_predictions.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

If the training went well, you should see both curves dropping rapidly in the first 20 epochs and then leveling off. The fact that the validation loss closely tracks the training loss tells us the model is not overfitting — it has genuinely learned the physics, not just memorized the data.

This is exactly what we want.

### 4.4 Testing Predictions Against Reality

The true test of our world model is this: can it accurately predict what CartPole will do over multiple steps? Let us take a real trajectory from the environment and compare it with what our model *thinks* will happen.

In [ ]:
def compare_predictions(model, env_name="CartPole-v1", max_steps=50):
    """
    Run a trajectory in the real environment and compare
    with multi-step predictions from the world model.
    """
    env = gym.make(env_name)
    state, _ = env.reset(seed=99)

    real_states = [state.copy()]
    model_states = [state.copy()]
    action_log = []

    # Use the same fixed action sequence for both
    current_model_state = torch.FloatTensor(state).unsqueeze(0)

    model.eval()
    for step in range(max_steps):
        action = env.action_space.sample()
        action_log.append(action)

        # Real environment step
        next_state, _, terminated, truncated, _ = env.step(action)
        real_states.append(next_state.copy())

        if terminated or truncated:
            break

        # World model prediction (autoregressive — feeds its own predictions back)
        action_tensor = torch.FloatTensor([[action]])
        with torch.no_grad():
            predicted_next = model(current_model_state, action_tensor)
        current_model_state = predicted_next
        model_states.append(predicted_next.squeeze().numpy())

        state = next_state

    env.close()
    return np.array(real_states), np.array(model_states), action_log

real_states, model_states, action_log = compare_predictions(world_model)
print(f"Trajectory length: {len(real_states)} steps")

In [ ]:
# 📊 Visualization: Predicted vs Actual States
fig, axes = plt.subplots(2, 2, figsize=(14, 8))
labels = ['Cart Position', 'Cart Velocity', 'Pole Angle', 'Pole Angular Velocity']
n_steps = min(len(real_states), len(model_states))

for i, (ax, label) in enumerate(zip(axes.flat, labels)):
    ax.plot(real_states[:n_steps, i], label='Real Environment',
            color='steelblue', linewidth=2)
    ax.plot(model_states[:n_steps, i], label='World Model Prediction',
            color='coral', linewidth=2, linestyle='--')
    ax.set_xlabel('Time Step', fontsize=11)
    ax.set_ylabel(label, fontsize=11)
    ax.set_title(label, fontsize=13, fontweight='bold')
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)

plt.suptitle('Real Environment vs World Model Predictions (Multi-Step)',
             fontsize=15, fontweight='bold')
plt.tight_layout()
plt.show()

Take a moment to study these plots. In the first few steps, the predicted and actual trajectories are nearly identical — the world model has learned the local physics accurately. As we go further into the future, small prediction errors accumulate and the trajectories may diverge. This is called **compounding error** and it is the fundamental challenge of learned world models.

But here is the key insight: for planning, we do not need perfect long-horizon predictions. We only need the predictions to be accurate enough over a short planning horizon (say 10-20 steps) to distinguish good action sequences from bad ones.

In [ ]:
#@title 🎧 Listen: Your Turn — Reward Predictor
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/12_todo1_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 5. 🔧 Your Turn

Now it is your turn. You have seen how to build a world model that predicts next states. In this section, you will extend it with two critical components:

1. A **reward predictor** — so the world model knows what is good and bad
2. A **random shooting planner** — so the agent can use imagination to make decisions

### TODO 1: Build a Reward Predictor

Your task is to build a neural network that predicts the reward given a state and action. In CartPole, the reward is 1.0 for every step the pole stays upright and the episode has not ended. The reward predictor should learn to approximate this.

In [ ]:
class RewardPredictor(nn.Module):
    """
    Predicts the reward given current state and action.

    Architecture: Linear(5, 64) -> ReLU -> Linear(64, 64) -> ReLU -> Linear(64, 1)

    Args:
        state_dim: dimension of the state vector (4 for CartPole)
        action_dim: dimension of the action (1 for CartPole)
        hidden_dim: number of hidden units
    """

    def __init__(self, state_dim=4, action_dim=1, hidden_dim=64):
        super().__init__()
        # ============ TODO ============
        # Build a neural network similar to WorldModel but with output dim = 1
        # Step 1: Create a nn.Sequential with:
        #   - Linear layer from (state_dim + action_dim) to hidden_dim
        #   - ReLU activation
        #   - Linear layer from hidden_dim to hidden_dim
        #   - ReLU activation
        #   - Linear layer from hidden_dim to 1
        #   - Sigmoid activation (rewards in CartPole are 0 or 1)
        # ==============================

        self.network = ???  # YOUR CODE HERE

    def forward(self, state, action):
        """
        Predict reward for a state-action pair.

        Args:
            state: (batch, 4) current state
            action: (batch, 1) action taken
        Returns:
            reward: (batch, 1) predicted reward
        """
        x = torch.cat([state, action], dim=-1)
        return self.network(x)

In [ ]:
# ✅ Verification: Run this cell to check your RewardPredictor implementation
reward_model = RewardPredictor()

# Test with a dummy input
test_state = torch.randn(8, 4)
test_action = torch.randint(0, 2, (8, 1)).float()
test_output = reward_model(test_state, test_action)

assert test_output.shape == (8, 1), \
    f"Expected output shape (8, 1), got {test_output.shape}"
assert (test_output >= 0).all() and (test_output <= 1).all(), \
    f"Expected outputs in [0, 1] (sigmoid), got min={test_output.min():.3f}, max={test_output.max():.3f}"

param_count = sum(p.numel() for p in reward_model.parameters())
assert param_count > 0, "Model has no parameters — did you define the network?"

print("✅ Correct! Your RewardPredictor architecture is valid.")
print(f"   Output shape: {test_output.shape}")
print(f"   Output range: [{test_output.min():.3f}, {test_output.max():.3f}]")
print(f"   Parameters: {param_count:,}")

In [ ]:
#@title 🎧 Listen: After TODO 1
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/13_after_todo1.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

Now let us train your reward predictor on the data we already collected.

In [ ]:
# Train the reward predictor (uses the same data we collected earlier)
def train_reward_model(model, states, actions, rewards, epochs=50, lr=1e-3):
    """Train the reward predictor using binary cross-entropy loss."""
    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.BCELoss()

    X_state = torch.FloatTensor(states)
    X_action = torch.FloatTensor(actions).unsqueeze(-1)
    Y = torch.FloatTensor(rewards).unsqueeze(-1)

    losses = []
    for epoch in range(epochs):
        pred = model(X_state, X_action)
        loss = loss_fn(pred, Y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append(loss.item())

        if (epoch + 1) % 10 == 0:
            accuracy = ((pred > 0.5).float() == Y).float().mean()
            print(f"Epoch {epoch+1:3d} | Loss: {loss.item():.4f} | Accuracy: {accuracy:.2%}")

    return losses

reward_losses = train_reward_model(reward_model, states, actions, rewards)
print("\nReward predictor training complete!")

In [ ]:
#@title 🎧 Listen: Your Turn — Random Shooting Planner
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/14_todo2_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### TODO 2: Build a Random Shooting Planner

Now the exciting part — let us use our trained world model and reward predictor to **plan.** The idea is simple:

1. From the current real state, sample $N$ random action sequences of length $H$
2. For each sequence, roll it forward through the world model to predict the trajectory
3. Use the reward predictor to estimate the total reward for each trajectory
4. Pick the action sequence with the highest predicted reward
5. Execute only the first action, then re-plan

This is the random shooting algorithm we described in Section 3.3.

In [ ]:
def random_shooting_planner(world_model, reward_model, current_state,
                            num_sequences=500, horizon=15, action_dim=2):
    """
    Plan using random shooting with the learned world model.

    Args:
        world_model: trained WorldModel that predicts next states
        reward_model: trained RewardPredictor that predicts rewards
        current_state: numpy array of shape (4,) — current real state
        num_sequences: N — how many random action sequences to evaluate
        horizon: H — how many steps to look ahead
        action_dim: number of possible actions (2 for CartPole: left/right)

    Returns:
        best_action: int — the first action of the best sequence
    """
    world_model.eval()
    reward_model.eval()

    with torch.no_grad():
        # ============ TODO ============
        # Step 1: Sample N random action sequences of length H
        #   - Each action is 0 or 1 (for CartPole)
        #   - Shape should be (num_sequences, horizon)
        #   Hint: use torch.randint

        action_sequences = ???  # YOUR CODE HERE

        # Step 2: Initialize states — repeat current_state N times
        #   - Convert current_state to a tensor of shape (num_sequences, 4)
        #   Hint: use torch.FloatTensor and .unsqueeze(0).repeat(num_sequences, 1)

        sim_states = ???  # YOUR CODE HERE

        # Step 3: Roll forward and accumulate rewards
        total_rewards = torch.zeros(num_sequences)

        for t in range(horizon):
            # Get the action for this timestep: shape (num_sequences, 1)
            actions_t = action_sequences[:, t].unsqueeze(-1).float()

            # Predict next state using world model
            # Hint: sim_states = world_model(sim_states, actions_t)

            ???  # YOUR CODE HERE

            # Predict reward using reward model
            # Hint: predicted_reward = reward_model(sim_states, actions_t)

            ???  # YOUR CODE HERE

            # Accumulate reward
            # Hint: total_rewards += predicted_reward.squeeze()

            ???  # YOUR CODE HERE

        # Step 4: Find the best action sequence
        # Hint: use torch.argmax on total_rewards

        best_idx = ???  # YOUR CODE HERE
        best_action = action_sequences[best_idx, 0].item()
        # ==============================

    return best_action

In [ ]:
# ✅ Verification: Run this cell to test your planner

# Test the planner on a simple state (pole nearly upright)
test_state = np.array([0.0, 0.0, 0.01, 0.0])  # Nearly balanced
best_action = random_shooting_planner(world_model, reward_model, test_state)

assert best_action in [0, 1], f"Expected action 0 or 1, got {best_action}"
print(f"✅ Planner returned action: {best_action}")

# Test that it runs without errors on multiple states
for s in [np.array([0.0, 0.0, 0.05, 0.3]),
          np.array([0.0, 0.0, -0.05, -0.3]),
          np.array([0.5, 0.1, 0.0, 0.0])]:
    a = random_shooting_planner(world_model, reward_model, s)
    assert a in [0, 1]

print("✅ Planner works correctly on all test states!")

In [ ]:
#@title 🎧 Listen: Putting It All Together
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/15_putting_together.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 6. Putting It All Together

Now we have all the pieces of the puzzle ready. Let us assemble the full pipeline:

1. **Collect data** from the real environment (done)
2. **Train a world model** to predict next states (done)
3. **Train a reward predictor** to predict rewards (done)
4. **Use random shooting** to plan actions (done)

The following function runs our model-based agent in the real CartPole environment. At every step, instead of taking a random action, the agent uses the planner to imagine 500 possible futures and picks the best one.

In [ ]:
def run_model_based_agent(world_model, reward_model, num_episodes=10,
                          max_steps=500, num_sequences=500, horizon=15):
    """
    Run the model-based planning agent in the real environment.

    Returns:
        episode_rewards: list of total rewards per episode
    """
    env = gym.make("CartPole-v1")
    episode_rewards = []

    for ep in range(num_episodes):
        state, _ = env.reset(seed=ep + 1000)
        total_reward = 0

        for step in range(max_steps):
            # Plan using imagination
            action = random_shooting_planner(
                world_model, reward_model, state,
                num_sequences=num_sequences, horizon=horizon
            )

            # Execute in real environment
            state, reward, terminated, truncated, _ = env.step(action)
            total_reward += reward

            if terminated or truncated:
                break

        episode_rewards.append(total_reward)
        print(f"Episode {ep+1:2d} | Reward: {total_reward:6.1f} | Steps: {step+1}")

    env.close()
    return episode_rewards

We also need a baseline to compare against. Let us run a purely random agent — one that takes actions without any planning.

In [ ]:
def run_random_agent(num_episodes=10, max_steps=500):
    """Run a random agent for comparison."""
    env = gym.make("CartPole-v1")
    episode_rewards = []

    for ep in range(num_episodes):
        state, _ = env.reset(seed=ep + 1000)
        total_reward = 0

        for step in range(max_steps):
            action = env.action_space.sample()
            state, reward, terminated, truncated, _ = env.step(action)
            total_reward += reward
            if terminated or truncated:
                break

        episode_rewards.append(total_reward)

    env.close()
    return episode_rewards

# Run the random baseline
print("Running random agent...")
random_rewards = run_random_agent()
print(f"\nRandom agent — Mean reward: {np.mean(random_rewards):.1f} +/- {np.std(random_rewards):.1f}")

In [ ]:
#@title 🎧 Listen: Results
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/16_results.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 7. Training and Results

Now let us run our model-based agent and see how imagination compares to random action.

In [ ]:
print("Running model-based planning agent...")
print("(This uses the world model to imagine futures at every step)\n")
planned_rewards = run_model_based_agent(world_model, reward_model)
print(f"\nModel-based agent — Mean reward: {np.mean(planned_rewards):.1f} +/- {np.std(planned_rewards):.1f}")

In [ ]:
# 📊 Visualization: Random vs Model-Based Agent Performance
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart comparison
episodes = range(1, len(random_rewards) + 1)
width = 0.35
axes[0].bar([e - width/2 for e in episodes], random_rewards, width,
            label='Random Agent', color='lightcoral', edgecolor='black')
axes[0].bar([e + width/2 for e in episodes], planned_rewards, width,
            label='Planning Agent', color='steelblue', edgecolor='black')
axes[0].set_xlabel('Episode', fontsize=12)
axes[0].set_ylabel('Total Reward', fontsize=12)
axes[0].set_title('Episode Rewards: Random vs Planning', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3, axis='y')

# Summary statistics
categories = ['Random\nAgent', 'Planning\nAgent']
means = [np.mean(random_rewards), np.mean(planned_rewards)]
stds = [np.std(random_rewards), np.std(planned_rewards)]
colors = ['lightcoral', 'steelblue']

bars = axes[1].bar(categories, means, yerr=stds, capsize=8,
                   color=colors, edgecolor='black', linewidth=1.2)
axes[1].set_ylabel('Mean Reward', fontsize=12)
axes[1].set_title('Average Performance Comparison', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar, mean, std in zip(bars, means, stds):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + std + 2,
                 f'{mean:.1f}', ha='center', fontsize=13, fontweight='bold')

plt.tight_layout()
plt.show()

# Print improvement
improvement = np.mean(planned_rewards) / max(np.mean(random_rewards), 1)
print(f"\nImprovement factor: {improvement:.1f}x")

Take a moment to appreciate what just happened. The planning agent has **never been trained with reinforcement learning.** It has never received a gradient signal telling it which actions are good. Instead, it learned a model of the world and uses that model to *think ahead* — to simulate possible futures and pick the best one.

The random agent stumbles around blindly, averaging roughly 20-30 steps before the pole falls. The planning agent, by imagining 500 possible futures at every step and picking the most promising one, keeps the pole balanced for significantly longer.

This is the power of world models: **learning the rules of the game lets you plan, rather than just react.**

In [ ]:
#@title 🎧 Listen: Final Output
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/17_final_output.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 8. 🎯 Final Output

Let us run one final side-by-side comparison to clearly see the difference between an agent that imagines and one that does not.

In [ ]:
def visualize_episode(world_model, reward_model, seed=42):
    """
    Run and visualize one episode each for random and planning agents.
    Shows state trajectories side-by-side.
    """
    env = gym.make("CartPole-v1")

    # Random agent trajectory
    state, _ = env.reset(seed=seed)
    random_traj = [state.copy()]
    for _ in range(500):
        action = env.action_space.sample()
        state, _, terminated, truncated, _ = env.step(action)
        random_traj.append(state.copy())
        if terminated or truncated:
            break

    # Planning agent trajectory
    state, _ = env.reset(seed=seed)
    planned_traj = [state.copy()]
    for _ in range(500):
        action = random_shooting_planner(world_model, reward_model, state)
        state, _, terminated, truncated, _ = env.step(action)
        planned_traj.append(state.copy())
        if terminated or truncated:
            break

    env.close()

    random_traj = np.array(random_traj)
    planned_traj = np.array(planned_traj)

    # Plot
    fig, axes = plt.subplots(2, 2, figsize=(14, 9))
    labels = ['Cart Position', 'Cart Velocity', 'Pole Angle', 'Pole Angular Velocity']

    for i, (ax, label) in enumerate(zip(axes.flat, labels)):
        ax.plot(random_traj[:, i], label=f'Random ({len(random_traj)} steps)',
                color='lightcoral', linewidth=2, alpha=0.8)
        ax.plot(planned_traj[:, i], label=f'Planning ({len(planned_traj)} steps)',
                color='steelblue', linewidth=2)
        ax.set_xlabel('Time Step', fontsize=11)
        ax.set_ylabel(label, fontsize=11)
        ax.set_title(label, fontsize=13, fontweight='bold')
        ax.legend(fontsize=10)
        ax.grid(True, alpha=0.3)

    plt.suptitle('Single Episode: Random Agent vs World Model Planning Agent',
                 fontsize=15, fontweight='bold')
    plt.tight_layout()
    plt.show()

    return len(random_traj), len(planned_traj)

random_len, planned_len = visualize_episode(world_model, reward_model)

In [ ]:
# 🎯 Final summary
print("=" * 60)
print("   WORLD MODELS FROM FIRST PRINCIPLES — COMPLETE!")
print("=" * 60)
print()
print("What you built today:")
print(f"  1. A neural network world model ({sum(p.numel() for p in world_model.parameters()):,} parameters)")
print(f"  2. A reward predictor ({sum(p.numel() for p in reward_model.parameters()):,} parameters)")
print(f"  3. A random shooting planner (500 imagined futures per step)")
print()
print("Results:")
print(f"  Random agent:   {np.mean(random_rewards):5.1f} avg reward")
print(f"  Planning agent: {np.mean(planned_rewards):5.1f} avg reward")
print(f"  Improvement:    {np.mean(planned_rewards)/max(np.mean(random_rewards),1):.1f}x")
print()
print("The key idea: the planning agent NEVER learned a policy directly.")
print("It learned the RULES of the world, then used imagination to act.")
print()
print("Congratulations — you have built a world model from scratch!")
print("=" * 60)

In [ ]:
#@title 🎧 Listen: Closing & Next Steps
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/18_closing.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 9. Reflection and Next Steps

### 🤔 Reflection Questions

Take a few minutes to think about these questions. They will deepen your understanding and prepare you for the next notebook.

1. **Compounding error:** We saw that multi-step predictions diverge from reality over time. What would happen if we increased the planning horizon from 15 to 100 steps? Would the agent perform better or worse? Why?

2. **Data distribution:** We collected training data using a *random* policy. What would happen if we trained the world model on data from a *good* policy instead? Would the model be better or worse at predicting what happens when the agent makes mistakes?

3. **Model capacity:** Our world model has just 2 hidden layers with 128 units. CartPole has simple physics. What would need to change if we wanted to model a more complex environment like a robotic arm with 20 joints?

### Optional Challenges

**Challenge 1: Improve the planner.** The random shooting planner is the simplest possible planning algorithm. Try implementing the **Cross-Entropy Method (CEM)**: instead of sampling completely random sequences, sample from a distribution, evaluate them, keep the top 10%, and refit the distribution. Repeat for 3-5 iterations. Does this improve performance?

**Challenge 2: Learn from imagination.** Right now, we plan at every step, which is slow. Can you use the world model to generate *synthetic training data* — imagined trajectories — and then train a simple policy network on that data? This is the core idea behind the Dyna architecture (Sutton, 1991).

---

**Coming up in Notebook 2:** We will move beyond simple feedforward models and explore how Ha and Schmidhuber's *"World Models"* paper (2018) used a **VAE + RNN** architecture to learn compressed, temporal world models — enabling an agent to learn entirely inside its own dream.

*This notebook is part of the Vizuara series: World Action Models — How AI Learns to Imagine Before It Acts.*

In [ ]:
#@title 💬 AI Teaching Assistant — Click ▶ to start
#@markdown This AI chatbot reads your notebook and can answer questions about any concept, code, or exercise.

import json as _json
import requests as _requests
from google.colab import output as _output
from IPython.display import display, HTML as _HTML, Markdown as _Markdown

# --- Read notebook content for context ---
def _get_notebook_context():
    try:
        from google.colab import _message
        nb = _message.blocking_request("get_ipynb", request="", timeout_sec=10)
        cells = nb.get("ipynb", {}).get("cells", [])
        parts = []
        for cell in cells:
            src = "".join(cell.get("source", []))
            tags = cell.get("metadata", {}).get("tags", [])
            if "chatbot" in tags:
                continue
            if src.strip():
                ct = cell.get("cell_type", "unknown")
                parts.append(f"[{ct.upper()}]\n{src}")
        return "\n\n---\n\n".join(parts)
    except Exception:
        return "Notebook content unavailable."

_NOTEBOOK_CONTEXT = _get_notebook_context()
_CHAT_HISTORY = []
_API_URL = "https://course-creator-brown.vercel.app/api/chat"

def _notebook_chat(question):
    global _CHAT_HISTORY
    try:
        resp = _requests.post(_API_URL, json={
            'question': question,
            'context': _NOTEBOOK_CONTEXT[:100000],
            'history': _CHAT_HISTORY[-10:],
        }, timeout=60)
        data = resp.json()
        answer = data.get('answer', 'Sorry, I could not generate a response.')
        _CHAT_HISTORY.append({'role': 'user', 'content': question})
        _CHAT_HISTORY.append({'role': 'assistant', 'content': answer})
        return answer
    except Exception as e:
        return f'Error connecting to teaching assistant: {str(e)}'

_output.register_callback('notebook_chat', _notebook_chat)

def ask(question):
    """Ask the AI teaching assistant a question about this notebook."""
    answer = _notebook_chat(question)
    display(_Markdown(answer))

print("\u2705 AI Teaching Assistant is ready!")
print("\U0001f4a1 Use the chat below, or call ask(\'your question\') in any cell.")

# --- Display chat widget ---
display(_HTML('''<style>
  .vc-wrap{font-family:-apple-system,BlinkMacSystemFont,'Segoe UI',Roboto,sans-serif;max-width:100%;border-radius:16px;overflow:hidden;box-shadow:0 4px 24px rgba(0,0,0,.12);background:#fff;border:1px solid #e5e7eb}
  .vc-hdr{background:linear-gradient(135deg,#667eea 0%,#764ba2 100%);color:#fff;padding:16px 20px;display:flex;align-items:center;gap:12px}
  .vc-avatar{width:42px;height:42px;background:rgba(255,255,255,.2);border-radius:50%;display:flex;align-items:center;justify-content:center;font-size:22px}
  .vc-hdr h3{font-size:16px;font-weight:600;margin:0}
  .vc-hdr p{font-size:12px;opacity:.85;margin:2px 0 0}
  .vc-msgs{height:420px;overflow-y:auto;padding:16px;background:#f8f9fb;display:flex;flex-direction:column;gap:10px}
  .vc-msg{display:flex;flex-direction:column;animation:vc-fade .25s ease}
  .vc-msg.user{align-items:flex-end}
  .vc-msg.bot{align-items:flex-start}
  .vc-bbl{max-width:85%;padding:10px 14px;border-radius:16px;font-size:14px;line-height:1.55;word-wrap:break-word}
  .vc-msg.user .vc-bbl{background:linear-gradient(135deg,#667eea 0%,#764ba2 100%);color:#fff;border-bottom-right-radius:4px}
  .vc-msg.bot .vc-bbl{background:#fff;color:#1a1a2e;border:1px solid #e8e8e8;border-bottom-left-radius:4px}
  .vc-bbl code{background:rgba(0,0,0,.07);padding:2px 6px;border-radius:4px;font-size:13px;font-family:'Fira Code',monospace}
  .vc-bbl pre{background:#1e1e2e;color:#cdd6f4;padding:12px;border-radius:8px;overflow-x:auto;margin:8px 0;font-size:13px}
  .vc-bbl pre code{background:none;padding:0;color:inherit}
  .vc-bbl h3,.vc-bbl h4{margin:10px 0 4px;font-size:15px}
  .vc-bbl ul,.vc-bbl ol{margin:4px 0;padding-left:20px}
  .vc-bbl li{margin:2px 0}
  .vc-chips{display:flex;flex-wrap:wrap;gap:8px;padding:0 16px 12px;background:#f8f9fb}
  .vc-chip{background:#fff;border:1px solid #d1d5db;border-radius:20px;padding:6px 14px;font-size:12px;cursor:pointer;transition:all .15s;color:#4b5563}
  .vc-chip:hover{border-color:#667eea;color:#667eea;background:#f0f0ff}
  .vc-input{display:flex;padding:12px 16px;background:#fff;border-top:1px solid #eee;gap:8px}
  .vc-input input{flex:1;padding:10px 16px;border:2px solid #e8e8e8;border-radius:24px;font-size:14px;outline:none;transition:border-color .2s}
  .vc-input input:focus{border-color:#667eea}
  .vc-input button{background:linear-gradient(135deg,#667eea 0%,#764ba2 100%);color:#fff;border:none;border-radius:50%;width:42px;height:42px;cursor:pointer;display:flex;align-items:center;justify-content:center;font-size:18px;transition:transform .1s}
  .vc-input button:hover{transform:scale(1.05)}
  .vc-input button:disabled{opacity:.5;cursor:not-allowed;transform:none}
  .vc-typing{display:flex;gap:5px;padding:4px 0}
  .vc-typing span{width:8px;height:8px;background:#667eea;border-radius:50%;animation:vc-bounce 1.4s infinite ease-in-out}
  .vc-typing span:nth-child(2){animation-delay:.2s}
  .vc-typing span:nth-child(3){animation-delay:.4s}
  @keyframes vc-bounce{0%,80%,100%{transform:scale(0)}40%{transform:scale(1)}}
  @keyframes vc-fade{from{opacity:0;transform:translateY(8px)}to{opacity:1;transform:translateY(0)}}
  .vc-note{text-align:center;font-size:11px;color:#9ca3af;padding:8px 16px 12px;background:#fff}
</style>
<div class="vc-wrap">
  <div class="vc-hdr">
    <div class="vc-avatar">&#129302;</div>
    <div>
      <h3>Vizuara Teaching Assistant</h3>
      <p>Ask me anything about this notebook</p>
    </div>
  </div>
  <div class="vc-msgs" id="vcMsgs">
    <div class="vc-msg bot">
      <div class="vc-bbl">&#128075; Hi! I've read through this entire notebook. Ask me about any concept, code block, or exercise &mdash; I'm here to help you learn!</div>
    </div>
  </div>
  <div class="vc-chips" id="vcChips">
    <span class="vc-chip" onclick="vcAsk(this.textContent)">Explain the main concept</span>
    <span class="vc-chip" onclick="vcAsk(this.textContent)">Help with the TODO exercise</span>
    <span class="vc-chip" onclick="vcAsk(this.textContent)">Summarize what I learned</span>
  </div>
  <div class="vc-input">
    <input type="text" id="vcIn" placeholder="Ask about concepts, code, exercises..." />
    <button id="vcSend" onclick="vcSendMsg()">&#10148;</button>
  </div>
  <div class="vc-note">AI-generated &middot; Verify important information &middot; <a href="#" onclick="vcClear();return false" style="color:#667eea">Clear chat</a></div>
</div>
<script>
(function(){
  var msgs=document.getElementById('vcMsgs'),inp=document.getElementById('vcIn'),
      btn=document.getElementById('vcSend'),chips=document.getElementById('vcChips');

  function esc(s){var d=document.createElement('div');d.textContent=s;return d.innerHTML}

  function md(t){
    return t
      .replace(/```(\w*)\n([\s\S]*?)```/g,function(_,l,c){return '<pre><code>'+esc(c)+'</code></pre>'})
      .replace(/`([^`]+)`/g,'<code>$1</code>')
      .replace(/\*\*([^*]+)\*\*/g,'<strong>$1</strong>')
      .replace(/\*([^*]+)\*/g,'<em>$1</em>')
      .replace(/^#### (.+)$/gm,'<h4>$1</h4>')
      .replace(/^### (.+)$/gm,'<h4>$1</h4>')
      .replace(/^## (.+)$/gm,'<h3>$1</h3>')
      .replace(/^\d+\. (.+)$/gm,'<li>$1</li>')
      .replace(/^- (.+)$/gm,'<li>$1</li>')
      .replace(/\n\n/g,'<br><br>')
      .replace(/\n/g,'<br>');
  }

  function addMsg(text,isUser){
    var m=document.createElement('div');m.className='vc-msg '+(isUser?'user':'bot');
    var b=document.createElement('div');b.className='vc-bbl';
    b.innerHTML=isUser?esc(text):md(text);
    m.appendChild(b);msgs.appendChild(m);msgs.scrollTop=msgs.scrollHeight;
  }

  function showTyping(){
    var m=document.createElement('div');m.className='vc-msg bot';m.id='vcTyping';
    m.innerHTML='<div class="vc-bbl"><div class="vc-typing"><span></span><span></span><span></span></div></div>';
    msgs.appendChild(m);msgs.scrollTop=msgs.scrollHeight;
  }

  function hideTyping(){var e=document.getElementById('vcTyping');if(e)e.remove()}

  window.vcSendMsg=function(){
    var q=inp.value.trim();if(!q)return;
    inp.value='';chips.style.display='none';
    addMsg(q,true);showTyping();btn.disabled=true;
    google.colab.kernel.invokeFunction('notebook_chat',[q],{})
      .then(function(r){
        hideTyping();
        var a=r.data['application/json'];
        addMsg(typeof a==='string'?a:JSON.stringify(a),false);
      })
      .catch(function(){
        hideTyping();
        addMsg('Sorry, I encountered an error. Please check your internet connection and try again.',false);
      })
      .finally(function(){btn.disabled=false;inp.focus()});
  };

  window.vcAsk=function(q){inp.value=q;vcSendMsg()};
  window.vcClear=function(){
    msgs.innerHTML='<div class="vc-msg bot"><div class="vc-bbl">&#128075; Chat cleared. Ask me anything!</div></div>';
    chips.style.display='flex';
  };

  inp.addEventListener('keypress',function(e){if(e.key==='Enter')vcSendMsg()});
  inp.focus();
})();
</script>'''))